In [1]:
import os
import sys
print("Python version" + sys.version)
os.getcwd()
print(sys.executable)

import numpy as np

import pandas as pd
import umap
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
import seaborn as sns

Python version3.9.10 | packaged by conda-forge | (main, Feb  1 2022, 21:24:11) 
[GCC 9.4.0]
/fast/users/twei_m/work/miniconda/envs/sc/bin/python


2023-06-07 12:49:32.658907: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-07 12:49:32.664738: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-07 12:49:32.664762: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from pathlib import Path

In [3]:
samples = ['p007', 'p008', 'p009', 'p013', 'p014', 'p016', 'p020', 'p021', 'p026', 'p035']

In [4]:
new_data_folder = '/fast/users/twei_m/work/crc/datasets_new_preprocessing'

sep_samples = ['p007', 'p008', 'p009_1', 'p009_2', 'p013', 'p014', 'p016', 'p020', 'p021', 'p026', 'p035']

In [5]:
joint_post = pd.DataFrame(columns=['CHROM', 'seg', 'seg_start', 'seg_end', 'cnv_state', 'p_cnv'])

In [6]:
for sample in samples:
#for sample in sep_samples:

    cell_seg = pd.read_csv(Path(new_data_folder)/f'Numbat/{sample}/joint_post_2.tsv',header=0,index_col=0,sep='\t')
    
    cell_seg.index = pd.DataFrame(cell_seg.index)['cell'].str.replace('_',':')
    
    cell_seg['cell_id'] = cell_seg.index
    
    cell_seg = pd.DataFrame(cell_seg[['CHROM', 'seg', 'seg_start', 'seg_end', 'cnv_state', 'p_cnv', 'cell_id']])  
    
    joint_post = pd.concat([joint_post, cell_seg], ignore_index=True)
    

In [7]:
joint_post = joint_post.set_index('cell_id')

In [ ]:
# new data 242293 rows × 6 columns
# after CB: 408033 rows × 6 columns

In [8]:
joint_post #210907 segments # sep: 203064

,CHROM,seg,seg_start,seg_end,cnv_state,p_cnv
cell_id,,,,,,
p007t:TTACGCCGTCAGTCGC,1,1a,758351,248906235,loh,8.785014e-21
p007t:TTACGCCGTCAGTCGC,3,3b,2307295,198089694,del,8.480627e-37
p007t:TTACGCCGTCAGTCGC,4,4a,80239,189967209,del,1.101603e-13
p007t:TTACGCCGTCAGTCGC,5,5a,201435,21480002,del,1.000000e+00
p007t:TTACGCCGTCAGTCGC,5,5b,21497162,37288433,amp,5.100128e-01
...,...,...,...,...,...,...
p035t:CTTGATTAGAACCCGA,13,13a,19609889,114314503,amp,8.512330e-01
p035t:CTTGATTAGAACCCGA,22,22b,49594148,50783624,amp,4.982685e-01
p035t:TTTGGAGTCATGCTAG,8,8b,101632857,145056240,amp,5.684066e-01


In [9]:
joint_post.to_csv(Path(new_data_folder)/'Numbat/20230607_numbat_all_output_joint_post_combined9.csv')

joint_post.to_csv(Path(new_data_folder)/'Numbat/20230525_numbat_all_output_joint_post_combined9.csv')

joint_post.to_csv(os.path.join('20230321_numbat_all_output_joint_post_sep9.csv'))

### clones

In [10]:
clone_post = pd.DataFrame(columns=['compartment_opt', 'clone_opt','GT_opt', 'p_cnv'])

In [11]:
# for h5 object
for sample in samples:
#for sample in sep_samples:
    per_cell_clone = pd.read_csv(Path(new_data_folder)/f'Numbat/{sample}/clone_post_2.tsv',
                                 header=0,index_col=0,sep='\t')
    per_cell_clone.index = pd.DataFrame(per_cell_clone.index)['cell'].str.replace('_',':')
    per_cell_clone['cell_id'] = per_cell_clone.index
    per_cell_clone = pd.DataFrame(per_cell_clone[['compartment_opt', 'clone_opt','GT_opt', 'p_cnv', 'cell_id']])  
    
    clone_post = pd.concat([clone_post, per_cell_clone], ignore_index=True)
    

In [12]:
clone_post = clone_post.set_index('cell_id')

In [14]:
# new data 14115 rows × 4 columns 
# after CB 17623 rows × 4 columns

In [13]:
clone_post # 11347 # sep 11347

,compartment_opt,clone_opt,GT_opt,p_cnv
cell_id,,,,
p007t:AAACCCAGTCGGTGAA,normal,1,NaN,1.692342e-12
p007t:AAACCCAGTCTGCCTT,tumor,7,"13a,15a,5a,9a,21a,17a,22a",9.999551e-01
p007t:AAACCCATCGCATTAG,tumor,3,"13a,15a,5a,9a,21a,17a,11a",1.000000e+00
p007t:AAACGAACACCACATA,normal,1,NaN,6.954112e-50
p007t:AAACGAATCGCTCATC,normal,1,NaN,8.677122e-227
...,...,...,...,...
p035t:TTTGGAGTCATGCTAG,normal,1,NaN,2.935156e-01
p035t:TTTGGTTTCATAGCAC,tumor,3,"13a,8b",8.467798e-01
p035t:TTTGTTGAGTGTACAA,tumor,2,13a,7.820054e-01


In [15]:
clone_post.to_csv(Path(new_data_folder)/'Numbat/20230607_numbat_all_output_clone_post_combined9.csv')

clone_post.to_csv(Path(new_data_folder)/'Numbat/2023525_numbat_all_output_clone_post_combined9.csv')

clone_post.to_csv(os.path.join('20230321_numbat_all_output_clone_post_sep9.csv'))

#### how different are the results from init = 1 and init =2, 2 is the default of interpret numbat result
for sample in samples:
    post1 = pd.read_csv(os.path.join(sample, 'clone_post_1.tsv'),header=0,index_col=0,sep='\t')
    post2 = pd.read_csv(os.path.join(sample, 'clone_post_2.tsv'),header=0,index_col=0,sep='\t')
    print(post1.index.equals(post2.index),post1['compartment_opt'].equals(post2['compartment_opt']))